In [2]:
import numpy as np
import cvxpy as cvx
from numpy.random import randn

# Problem 1

In [17]:
n = 20
M = 25
K = 100
np.random.seed(1)
c_true = randn(n)
X  = randn(n,K)
y = np.dot(X.T, c_true) + 0.1 * np.sqrt(n) * randn(K)
print(c_true.shape)

(20,)


In [18]:
sort_ind = np.argsort(y)
X = X[:, sort_ind]
y = y[sort_ind[:M+1]]
D = (y[M-1] + y[M])/2
y = y[:M,np.newaxis]

Very simple treatment: about points X[:,M:] we know only that $y_i$ should be $y_i \geq D$, so let's have constraint $c^\top x_i \geq D, i > M$ and minimize by X[:,:M] part of X.
$$
\min_{X} c^\top X_{i \leq M}, \\
c^\top X_{i > M} \geq D
$$

In [58]:
c = cvx.Variable(n)
constraints = [c.T*X[:,M:] >= D] # our X[:,M:] is this part with censored values for y

In [59]:
obj = cvx.Minimize(cvx.sum_squares(c.T*X[:,:M]-y.T)) # and minimize by known y

In [60]:
prob = cvx.Problem(obj, constraints) #usual routine

In [61]:
prob = cvx.Problem(obj, constraints)
prob.solve()  
print ("status:", prob.status)
print ("optimal value %.7f" % (prob.value))
print(c.value)

status: optimal
optimal value 2.0711948
[[ 1.53817956]
 [-0.75310014]
 [-0.43716121]
 [-1.04458413]
 [ 0.87371472]
 [-2.28535139]
 [ 1.79176345]
 [-0.71101112]
 [ 0.2644144 ]
 [-0.30984617]
 [ 1.43450893]
 [-2.115688  ]
 [-0.35096751]
 [-0.37592943]
 [ 1.18146074]
 [-1.1473156 ]
 [-0.04681776]
 [-1.14158709]
 [ 0.13219842]
 [ 0.54257486]]


In [62]:
c_true

array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
       -2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 , -0.24937038,
        1.46210794, -2.06014071, -0.3224172 , -0.38405435,  1.13376944,
       -1.09989127, -0.17242821, -0.87785842,  0.04221375,  0.58281521])

In [66]:
def compare(c1, c_true):
    return np.sqrt(np.sum((np.squeeze(np.asarray(c1))-c_true)**2)/np.sum(c_true**2))

In [67]:
print(compare(c.value, c_true))

0.0788161641764


In [69]:
prob2 = cvx.Problem(obj, []) # now we just remove constraints about i >= M points, that's means we just minimize by first known points
prob2.solve()  
print ("status:", prob2.status)
print ("optimal value %.7f" % (prob2.value))
print(compare(c.value, c_true))

status: optimal
optimal value 0.9040512
0.227096701662
